### Heatmap Generation - Traffic accident hotspot prediction (2015-2020)

In [1]:
import os
import math
import warnings
from os import walk
import pandas as pd
from tqdm import tqdm
from PIL import Image
import seaborn as sns
import geopy.distance
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
from haversine import haversine, Unit

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

#### Methods to help in the generation of spatio-temporal heatmaps

In [ ]:
# Most of the heatmap creation functionalities were used/modified from jeffmur's geoLife repository

def createMap(location, cell_size=5):
    
    ## Calculate bounds
    sLat = float(location[0])
    nLat = float(location[1])
    wLon = float(location[2])
    eLon = float(location[3])

    # all four corners
    SE = [sLat, eLon]
    SW = [sLat, wLon]
    NE = [nLat, eLon]
    NW = [nLat, wLon]

    bounds = {"SE": SE, "SW": SW, "NE": NE, "NW": NW}

    # Calculate Distance
    # SW -> NW
    width = math.ceil(geopy.distance.geodesic(SW, NW).miles)

    # NW -> NE
    length = math.ceil(geopy.distance.geodesic(NW, NE).miles)
    if(length == 0):
        length = 2400 #Alaska
        
    # Image Dimensions
    l_pix = int(math.ceil(length / cell_size))
    w_pix = int(math.ceil(width / cell_size))
    
    # Step Size for Lat/Lon comparison
    # Max distance / num of pixels
    
    step_length = (nLat - sLat) / l_pix  #  Step Lenth
    step_width = (eLon - wLon) / w_pix  #  Step Width

    # Steps in degrees
    step = {"width": step_width, "length": step_length}

    # Calculated Width and Length of image
    pix = {"length": l_pix, "width": w_pix}

    return bounds, step, pix


def frequencyHeatmap(bounds, pix, step, stdf):
    # Generates a Frequency Matrix
    nLat = bounds["NE"][0]
    eLon = bounds["NE"][1]

    columns = pix["width"]
    rows = pix["length"]

    step_w = step["width"]
    step_l = step["length"]

    freq_heat = pd.DataFrame(0, index=range(rows+1), columns=range(columns+1))
    lonLat = stdf[stdf.columns[7:9]].to_numpy()
    maxVal = 0

    for location in lonLat:
        r = round((nLat - location[1]) / step_l)
        c = round((eLon - location[0]) / step_w)
        #print(r,c)
        if (c <= columns) and (c >= 0) and (r <= rows) and (r >= 0):
            freq_heat.loc[r, c] += 1

            if maxVal < freq_heat.loc[r, c]:
                maxVal = freq_heat.loc[r, c]
    return maxVal, freq_heat

def genFMprime(freq_heat):
    # Generates an image representation of the Frequency Matrix
    dim = freq_heat.shape
    #print("dim",dim)
    img = Image.new("RGB", (dim[0], dim[1]), color="red")
    pixels = img.load()

    for i in range(freq_heat.shape[0]): #iterate over rows
        for j in range(freq_heat.shape[1]): #iterate over columns
            #print(i,j)
            if(freq_heat.at[i, j] == 1):
                pixels[i, j] = (255, 255, 255)
            else:
                pixels[i, j] = (0, 0, 0)
    
    '''
    for row in freq_heat.itertuples():
        # Need row index for assignment
        for c in range(1, len(row)):
            # Capture data point at [row, column]
            data = row[c]
            freq = int(255 * data)
            pixels[row[0], c - 1] = (freq, freq, freq)
            '''
    #print(img.size)
    return img

def dropOutlyingData(df, boundingbox):
    # Remove data outside the bounding box of Beijing
    lat = boundingbox[0:2]
    lon = boundingbox[2:4]

    return df.loc[
        (df.longitude >= float(lon[0]))
        & (df.longitude <= float(lon[1]))
        & (df.latitude >= float(lat[0]))
        & (df.latitude <= float(lat[1]))
    ].reset_index(drop=True)

#### Load accident datasets from FARS database

In [2]:
# Load data
FARSpath = "/data/fiona123/ProjectData_TrafficFatality/"
FARS = ["FARS2015NationalCSV","FARS2016NationalCSV","FARS2017NationalCSV","FARS2018NationalCSV",
       "FARS2019NationalCSV", "FARS2020NationalCSV"]

d = {}
for i in FARS:
    print(i)
    files = next(walk(FARSpath+i))[2]
    for j in files:
        if ("accident" in j) or ("ACCIDENT" in j):
            print(j)
            d[i[4:8] + "_" + j[:-4]] = pd.read_csv(FARSpath + i + "/" + j,encoding='ISO8859-1')

FARS2015NationalCSV
accident.csv
FARS2016NationalCSV
accident.CSV
FARS2017NationalCSV
accident.CSV
FARS2018NationalCSV
accident.csv
FARS2019NationalCSV
accident.CSV
FARS2020NationalCSV
accident.CSV


#### Concat all accident datasets and create a heatmapid based on Date

In [5]:
names = []
for i in d.keys():
    temp_df = d[i]
    names.append(temp_df)
d['accident'] = pd.concat(names)
d['accident_heatmap'] = d['accident'][['LATITUDE', 'LONGITUD','YEAR','MONTH','DAY','STATENAME']]
d['accident_heatmap']['heatmapid'] = d['accident_heatmap']['YEAR'].astype(str) + '_' + d['accident_heatmap']['MONTH'].astype(str) + '_' + d['accident_heatmap']['DAY'].astype(str)
display(d['accident_heatmap'].head(5))

,LATITUDE,LONGITUD,YEAR,MONTH,DAY,STATENAME,heatmapid
0,33.878653,-87.325328,2015,1,1,Alabama,2015_1_1
1,34.910442,-86.908708,2015,1,1,Alabama,2015_1_1
2,32.142006,-85.758456,2015,1,1,Alabama,2015_1_1
3,31.439814,-85.510300,2015,1,4,Alabama,2015_1_4
4,31.319331,-85.515100,2015,1,7,Alabama,2015_1_7


#### Create directories to store heatmaps for each state

In [6]:
states = d['accident_heatmap'].STATENAME.unique()

# Creates 50 directories for each state
for i in tqdm(states):
    path = "/data/fiona123/Project_GeneratedData/" + str(i)
    print(i)
    if not os.path.exists(path):
        os.mkdir(path)

100%|██████████████████████████████████████████████████████████████████████| 51/51 [00:00<00:00, 44196.18it/s]

Alabama
Alaska
Arizona
Arkansas
California
Colorado
Connecticut
Delaware
District of Columbia
Florida
Georgia
Hawaii
Idaho
Illinois
Indiana
Iowa
Kansas
Kentucky
Louisiana
Maine
Maryland
Massachusetts
Michigan
Minnesota
Mississippi
Missouri
Montana
Nebraska
Nevada
New Hampshire
New Jersey
New Mexico
New York
North Carolina
North Dakota
Ohio
Oklahoma
Oregon
Pennsylvania
Rhode Island
South Carolina
South Dakota
Tennessee
Texas
Utah
Vermont
Virginia
Washington
West Virginia
Wisconsin
Wyoming


#### Create heatmaps and save in respective directories

In [43]:
CELL_SIZE = 10000
SQ_CELL = float(CELL_SIZE) * 0.00062137

for i in tqdm(states):
    print(i)
    app = Nominatim(user_agent="traffic")
    loc = app.geocode(str(i)).raw
    path = "/data/fiona123/Project_GeneratedData/" + str(i) 
    for j in d['accident_heatmap']['heatmapid'].unique():
        df_id = d['accident_heatmap'][d['accident_heatmap']['heatmapid'] == j]
        df_id["longitude"] = df_id["LONGITUD"] 
        df_id["latitude"] = df_id["LATITUDE"] 

        if(len(df_id) == 0):
            del df_id 
            continue
        else: 
            df_id = dropOutlyingData(df_id, loc["boundingbox"])
            bounds, step, pix = createMap(loc["boundingbox"], cell_size=SQ_CELL)
            maxVal, freq_heat = frequencyHeatmap(bounds, pix, step, df_id)
            freq_heat = freq_heat.T
            img = genFMprime(freq_heat)
            img = img.transpose(Image.FLIP_LEFT_RIGHT)
            img = img.convert("L")
            img.save(path + "/" + str(j) + '.png', "PNG")

  0%|                                                                                  | 0/51 [00:00<?, ?it/s]/data/fiona123/kats_jup/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/data/fiona123/kats_jup/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/data/fiona123/kats_jup/lib/python3.7/site-packages/ipykernel_launcher.py:32: DeprecationWarning: FLIP_LEFT_RIGHT is deprecated and will be removed in Pillow 

Alabama


  2%|█▍                                                                        | 1/51 [01:00<50:46, 60.93s/it]

Alaska


  4%|██▊                                                                    | 2/51 [17:35<8:18:26, 610.33s/it]

...

 96%|████████████████████████████████████████████████████████████████████▏  | 49/51 [1:18:31<02:17, 68.61s/it]

Wisconsin


 98%|█████████████████████████████████████████████████████████████████████▌ | 50/51 [1:19:47<01:10, 70.80s/it]

Wyoming


100%|███████████████████████████████████████████████████████████████████████| 51/51 [1:20:53<00:00, 95.17s/it]
